# Zero-shot mutant prediction For VHH

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import pandas as pd
from Bio import SeqIO
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["http_proxy"] = "http://g5:15777"
os.environ["https_proxy"] = "http://g5:15777"

def read_seq(seq_file):
    for record in SeqIO.parse(seq_file, "fasta"):
        return str(record.seq)

wild_type = f"../example_data/expr/argonaute.fasta"
sequence = read_seq(wild_type)

all_mutants = []
for i, wt in enumerate(sequence):
    for aa in "ACDEFGHIKLMNPQRSTVWY":
        if aa != wt:
            mutant = f"{wt}{i+1}{aa}"
            all_mutants.append(mutant)
df = pd.DataFrame({"mutant": all_mutants})

In [2]:
base_model_path = "AI4Protein/argonaute_model_"

for i in range(0, 5):
    model_path = f"{base_model_path}{i}"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForMaskedLM.from_pretrained(model_path, trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained("AI4Protein/Prime_690M", trust_remote_code=True)
    model.eval()
    model = model.to(device)

    tokenized = tokenizer(sequence, return_tensors="pt").to(device)
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)
        logits = output.logits[0, 1:-1, :].log_softmax(dim=-1)
    
    scores = []
    for mutant in tqdm(df["mutant"]):
        score = 0
        for sub_mutant in mutant.split(":"):
            wt, idx, mt = sub_mutant[0], int(sub_mutant[1:-1]) - 1, sub_mutant[-1]
            score += (logits[idx, tokenizer.get_vocab()[mt]] - logits[idx, tokenizer.get_vocab()[wt]]).item()
        scores.append(score)
    df[f"model_{i}"] = scores
    print(f"Model {i} done")

  0%|          | 0/14003 [00:00<?, ?it/s]

Model 0 done


  0%|          | 0/14003 [00:00<?, ?it/s]

Model 1 done


  0%|          | 0/14003 [00:00<?, ?it/s]

Model 2 done


  0%|          | 0/14003 [00:00<?, ?it/s]

Model 3 done


  0%|          | 0/14003 [00:00<?, ?it/s]

Model 4 done


In [3]:
aggregate_score = 0
for i in range(0, 5):
    # min-max
    df[f"model_{i}"] = (df[f"model_{i}"] - df[f"model_{i}"].min()) / (df[f"model_{i}"].max() - df[f"model_{i}"].min())
    aggregate_score += df[f"model_{i}"]
df["aggregate"] = aggregate_score
df = df.sort_values("aggregate", ascending=False)

In [4]:
df.head(50)

,mutant,model_0,model_1,model_2,model_3,model_4,aggregate
11377,W599S,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000
10260,N541A,0.997358,0.997325,0.997339,0.996760,0.997234,4.986015
5522,R291P,0.992918,0.992888,0.992900,0.992509,0.992801,4.964016
1281,C68L,0.990468,0.990417,0.990413,0.990187,0.990330,4.951816
11651,E614G,0.985880,0.985823,0.985849,0.985374,0.985746,4.928671
3801,C201D,0.982448,0.982449,0.982468,0.982201,0.982391,4.911957
2453,Y130D,0.980356,0.980386,0.980395,0.980014,0.980308,4.901459
13840,G729L,0.978578,0.978537,0.978556,0.978337,0.978504,4.892511
9109,C480L,0.976600,0.976619,0.976681,0.976565,0.976719,4.883184
11786,E621I,0.974442,0.974385,0.974395,0.974369,0.974388,4.871979
